In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer

In [2]:
data = pd.read_csv(r"../data/gd.csv")

In [3]:
data.head()

,Account_status,Duration_months,Credit_history,loan_Purpose,Credit_amount,Savings/Bonds_AC,Present_Employment_since,loan_wage_ratio,Sex,Marital_status,...,Assets/Physical_property,Age,Other_loans,Housing,Existing Credit,Job_status,Dependents,Telephone,Foreign_worker,Credit_risk
0,A11,6,A34,A43,1169,A65,A75,4,Male,single,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,Female,divorced/separated/married,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,Male,single,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,Male,single,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,Male,single,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [5]:
data.columns

Index(['Account_status', 'Duration_months', 'Credit_history', 'loan_Purpose',
       'Credit_amount', 'Savings/Bonds_AC', 'Present_Employment_since',
       'loan_wage_ratio', 'Sex', 'Marital_status', 'co-debtors', 'Tenure',
       'Assets/Physical_property', 'Age', 'Other_loans', 'Housing',
       'Existing Credit', 'Job_status', 'Dependents', 'Telephone',
       'Foreign_worker', 'Credit_risk'],
      dtype='object')

In [ ]:
ohe_columns = ["Account_status", ]

In [ ]:
ct = ColumnTransformer(
    transformers = [
        ("cat")
    ]
)